In [1]:
import torch

print(torch.version.cuda)  # '12.8'
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # should show your RTX 50xx

12.8
True
NVIDIA GeForce RTX 5060 Ti


In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.utils.embedding_functions import EmbeddingFunction


# ephemeral client for local development
client = chromadb.Client()
print("Created ChromaDB client:", client)

c:\Users\peter\OneDrive\Documents\GitHub\09-15-2025_discord-meeting-transcriptor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Created ChromaDB client: <chromadb.api.client.Client object at 0x000001990366F8B0>


# Finished Setup

Now we can test things

In [52]:
class EF(EmbeddingFunction):
    def __init__(self):
        self.m = SentenceTransformer("BAAI/bge-large-en-v1.5")

    def __call__(self, texts):
        return self.m.encode(texts, normalize_embeddings=True).tolist()

In [6]:
ef = EF()
coll = client.get_or_create_collection("meeting_chunks", embedding_function=ef)

In [13]:
def iter_collection(coll, include_embeddings=False, batch_size=1000, where=None, ids=None):
    """
    Yields records from a Chroma collection.
    Record: {"id","document","metadata"[,"embedding"]}
    """
    include = ["metadatas", "documents"]  # don't include "ids"
    if include_embeddings:
        include.append("embeddings")

    offset = 0
    while True:
        res = coll.get(limit=batch_size, offset=offset, include=include, where=where, ids=ids)
        id_list = res.get("ids", [])
        n = len(id_list)
        if n == 0:
            return

        docs = res.get("documents") or [None] * n
        metas = res.get("metadatas") or [None] * n
        embs = res.get("embeddings") if include_embeddings else None

        for i in range(n):
            rec = {"id": id_list[i], "document": docs[i], "metadata": metas[i]}
            if embs is not None:
                rec["embedding"] = embs[i]
            yield rec

        offset += n
        if n < batch_size:
            return


def list_collection(coll, **kwargs):
    return list(iter_collection(coll, **kwargs))


# Example:
items = list_collection(coll, include_embeddings=False, batch_size=500)
print(f"{len(items)} records")
if items:
    from pprint import pprint
    pprint(items[0])

1 records
{'document': 'my favorite animal is a cat! i have a cat named Stella.',
 'id': 'the-first-item',
 'metadata': {'source': 'test_case'}}


In [82]:

import uuid

target_strings = [
"Stargazing on crisp winter nights reveals Orion's belt, glittering nebulae, and planets drifting steadily, reminding us space is vast, indifferent, yet beautifully ordered by gravity and light.",
"Sear chicken thighs in butter until golden, deglaze with lemon and capers, then simmer gently; the sauce thickens, bright and briny, perfect over rice or roasted potatoes.",
"Design services as small, cohesive modules with clear contracts; add observability, idempotent handlers, and backpressure. Favor simplicity over cleverness, because production failures punish ambiguity more than missing features.",
"Arriving before sunrise, the market hums: bakers dust loaves, fishermen sort silver catch, and baristas steam milk, releasing that hopeful hiss promising warmth, breakfast, and easy conversation.",
"A patient midfield triangle controls tempo with short passes, drawing defenders out, then slicing a diagonal ball behind the line, creating space for a calm, clinical finish.",
"That chord change feels like sunlight breaking through blinds; the bass breathes, drums lean forward, and the singer lifts vowels carefully, letting time stretch without ever losing momentum.",
"Compost transforms kitchen scraps into dark, sweet soil; worms churn patiently, fungi thread networks, and seedlings respond with thicker stems, brighter leaves, and roots that explore confidently through loosened beds.",
"The archive smelled of dust and glue; letters revealed ordinary worries during extraordinary times, reminding me history survives not in speeches, but in grocery lists, receipts, and quiet promises.",
"Compound interest is patient arithmetic: reinvest modest gains, avoid catastrophic losses, and time multiplies discipline; markets punish urgency, reward consistency, and quietly move wealth toward those who plan.",
"Sleep is the cheapest performance enhancer; dim lights, keep a cool room, respect a steady schedule, and your mood, memory, cravings, and training numbers will all noticeably improve.",
"Winter is my favorite season",
"April has the softest rain",
"My birthday is in April",
"I met my best friend in grade 5",
]

# clear db first
print("Clearing collection...")
# Get all items and delete them by ID
all_items = list_collection(coll, include_embeddings=False)
if all_items:
    all_ids = [item["id"] for item in all_items]
    coll.delete(ids=all_ids)
print("Collection cleared.")

# create embedding vector
for target_string in target_strings:
    custom_entry_id = str(uuid.uuid4())
    print(f"Processing entry ID: {custom_entry_id}")

    target_vector = ef([target_string])[0]
    print("Target vector generated: dimensions: {}".format(len(target_vector)))

    # add vector to collection
    # check if already exists, if yes delete
    existing = list_collection(coll, ids=[custom_entry_id])
    if existing:
        print(f"Entry with ID {custom_entry_id} already exists. Deleting it.")
        coll.delete(ids=[custom_entry_id])

    coll.add(
        documents=[target_string],
        metadatas=[{"source": "test_case"}],
        ids=[custom_entry_id],
        embeddings=[target_vector]
    )
    print(f"Added entry with ID {custom_entry_id}")

Clearing collection...
Collection cleared.
Processing entry ID: 7254d76c-4df7-445b-af86-b85d7c4227b0
Target vector generated: dimensions: 384
Added entry with ID 7254d76c-4df7-445b-af86-b85d7c4227b0
Processing entry ID: b9010c7a-46fe-4b4f-bd23-29403225b87d
Target vector generated: dimensions: 384
Added entry with ID b9010c7a-46fe-4b4f-bd23-29403225b87d
Processing entry ID: 6b56c34d-28c7-468b-9dc3-71774295589d
Target vector generated: dimensions: 384
Added entry with ID 6b56c34d-28c7-468b-9dc3-71774295589d
Processing entry ID: 1c3cef63-fb4f-4a34-80ae-8778d66fa725
Target vector generated: dimensions: 384
Added entry with ID 1c3cef63-fb4f-4a34-80ae-8778d66fa725
Processing entry ID: 0ec763df-eee7-4869-a436-b6352afd2457
Target vector generated: dimensions: 384
Added entry with ID 0ec763df-eee7-4869-a436-b6352afd2457
Processing entry ID: 80a07fd5-c937-42a3-aded-8019998cfa8f
Target vector generated: dimensions: 384
Added entry with ID 80a07fd5-c937-42a3-aded-8019998cfa8f
Processing entry ID: 

In [33]:
def query_collection(coll, query_string, top_k=10):
    # create query vector
    query_vector = ef([query_string])[0]

    # perform similarity search
    results = coll.query(
        query_embeddings=[query_vector],
        n_results=top_k,
        include=["metadatas", "documents"]
    )
    return results


## Reranker using SentenceTransformer

In [51]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder('BAAI/bge-reranker-v2-m3')

def rerank(query, candidates, top_k=10):
    # Handle both flat lists and nested lists from ChromaDB
    if candidates and isinstance(candidates[0], list):
        candidates = candidates[0]
    
    pairs = [(query, c) for c in candidates]
    scores = reranker.predict(pairs)  # one score per pair
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return [c for c, _ in ranked[:top_k]]


### Testing

In [74]:


# query selection
query_1 = "what are some important dates?"
qdb_1 = query_collection(coll, query_1, top_k=5)
print(qdb_1)
rerank_1 = rerank(query_1, qdb_1['documents'], top_k=5)
print("Reranked results:")
for doc in rerank_1:
    print(doc)


    


{'ids': [['70251634-52f7-4e8d-82c5-807b63fbf2de', 'a8a75579-3850-4f9a-9736-d09ef9a276c2', 'ffa381b2-a3f5-4683-b0be-50ceb19d7c8a', '5299f266-b8e2-4804-8b58-c974b635c000', 'ec65ad44-fce2-439c-ba08-ab4debcec68c']], 'embeddings': None, 'documents': [['My birthday is in April', 'Compound interest is patient arithmetic: reinvest modest gains, avoid catastrophic losses, and time multiplies discipline; markets punish urgency, reward consistency, and quietly move wealth toward those who plan.', 'Winter is my favorite season', 'April has the softest rain', 'I met my best friend in grade 5']], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [[{'source': 'test_case'}, {'source': 'test_case'}, {'source': 'test_case'}, {'source': 'test_case'}, {'source': 'test_case'}]], 'distances': None}
Reranked results:
My birthday is in April
I met my best friend in grade 5
Winter is my favorite season
April has the softest rain
Compound interest is patient arithmetic: reinvest m